In [16]:
!pip install langchain openai langchain-openai chromadb tiktoken pypdf gptcache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 1.7 MB/s eta 0:00:00


In [17]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [52]:
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [53]:
template = """
  You are a chatbot having conversation with 5 year human child,
  {chat_history}
  Kid: {request_instruction}
  chatbot:
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "request_instruction"], template=template
)

In [54]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [56]:
# Build chain
llm_chain = LLMChain(
    llm=OpenAI(openai_api_key = OPENAI_API_KEY),
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [57]:
lang=input("Language?")

request_instruction=f"Good Morning in {lang}"

llm_chain.predict(request_instruction=request_instruction)

Language?Hindi


> Entering new LLMChain chain...
Prompt after formatting:
 
  You are a chatbot having conversation with 5 year human child,
  
  Kid: Good Morning in Hindi 
  chatbot:


> Finished chain.


'Good morning, in Hindi we say "suprabhat"! How are you doing today?'

In [61]:
lang=input("question?")
request_instruction=f"Good Morning in {lang}"

llm_chain.predict(request_instruction=request_instruction)

question?hindi


> Entering new LLMChain chain...
Prompt after formatting:
 
  You are a chatbot having conversation with 5 year human child,
  Human: Good Morning in Hindi
AI: Good morning, in Hindi we say "suprabhat"! How are you doing today?
Human: Good Morning in how was your day?
AI: 
My day has been great so far, thank you for asking! How about you, what have you been up to today?
Human: Good Morning in kannada
AI: 
In Kannada, we say "shubhodaya" for Good Morning. How are you doing today, my little friend?
Human:  what was the first message from me? 
AI:   Good morning, in Hindi we say "suprabhat"! How are you doing today?
  Kid: Good Morning in hindi 
  chatbot:


> Finished chain.


'\nGood morning, in Hindi we say "suprabhat"! How are you doing today?'

In [62]:
lang=input("question?")
request_instruction=f"How was your {lang}"

llm_chain.predict(request_instruction=request_instruction)

question?day


> Entering new LLMChain chain...
Prompt after formatting:
 
  You are a chatbot having conversation with 5 year human child,
  Human: Good Morning in Hindi
AI: Good morning, in Hindi we say "suprabhat"! How are you doing today?
Human: Good Morning in how was your day?
AI: 
My day has been great so far, thank you for asking! How about you, what have you been up to today?
Human: Good Morning in kannada
AI: 
In Kannada, we say "shubhodaya" for Good Morning. How are you doing today, my little friend?
Human:  what was the first message from me? 
AI:   Good morning, in Hindi we say "suprabhat"! How are you doing today?
Human: Good Morning in hindi
AI: 
Good morning, in Hindi we say "suprabhat"! How are you doing today?
  Kid: How was your day 
  chatbot:


> Finished chain.


'  My day has been great so far, thank you for asking! How about you, what have you been up to today?'

In [63]:
llm_chain.predict(request_instruction=" what was the first message from me? ")



> Entering new LLMChain chain...
Prompt after formatting:
 
  You are a chatbot having conversation with 5 year human child,
  Human: Good Morning in Hindi
AI: Good morning, in Hindi we say "suprabhat"! How are you doing today?
Human: Good Morning in how was your day?
AI: 
My day has been great so far, thank you for asking! How about you, what have you been up to today?
Human: Good Morning in kannada
AI: 
In Kannada, we say "shubhodaya" for Good Morning. How are you doing today, my little friend?
Human:  what was the first message from me? 
AI:   Good morning, in Hindi we say "suprabhat"! How are you doing today?
Human: Good Morning in hindi
AI: 
Good morning, in Hindi we say "suprabhat"! How are you doing today?
Human: How was your day
AI:   My day has been great so far, thank you for asking! How about you, what have you been up to today?
  Kid:  what was the first message from me?  
  chatbot:


> Finished chain.


'The first message from you was "Good Morning in Hindi". Is there anything else you would like to talk about?'

# Document Loader

In [106]:
# Load Document
loader = PyPDFLoader("/content/TheLetter.pdf")
documents = loader.load_and_split()

In [107]:
# Split the documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.split_documents(documents)

In [108]:
# Embeddings
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large",
                              openai_api_key=OPENAI_API_KEY)

In [109]:
# DB
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(texts, embeddings)

In [110]:
# expose this index in a retriever interface
retriever = db.as_retriever(
    search_type="similarity", search_kwargs={"k": 2}
)

In [111]:
# Prompt template
prompt_template = """
        You are a Chat customer support agent.\
        Address the customer as Dear Mr. or Miss. depending on customer's gender followed by Customer's First Name.\
        Use the following pieces of context to answer the question at the end.\
        If you don't know the answer, just say that you don't know, don't try to make up an answer.\
        Below are the details of the customer:\
        Customer's Name : {Customer_Name} \
        Customer's Resident State: {Customer_State}\
        Customer's Gender: {Customer_Gender}\
        {context}\
        Question: {question}\
        {chat_history}\
        Answer:
        """

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question","Customer_Name","Customer_State","Customer_Gender"]
)

In [112]:
# Memory
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question")

In [113]:
# Chain
from langchain.chains.question_answering import load_qa_chain


chain = load_qa_chain(
    OpenAI(temperature = 0 , openai_api_key = OPENAI_API_KEY),
    chain_type="stuff",
    memory=memory,
    prompt=PROMPT,

)

In [114]:
query="why frog sent letter to Todd??"

docs = db.similarity_search(query=query)

# building the dictionary for chain

chain_input={
    "input_documents": docs,
    "context":"This is contextless",
    "question":query,
    "Customer_Name":"Bob",
    "Customer_State":"NY",
    "Customer_Gender":"Male"
}

result=chain.invoke(chain_input, return_only_outputs=True)

In [115]:
result['output_text']

'\nFrog sent a letter to Toad because he wanted to show his appreciation for their friendship. He wrote, "Dear Toad, I am glad that you are my best friend. Frog." Frog wanted to make Toad happy and show him that he is loved and valued.'

In [116]:
query="What was the last question?"

docs = db.similarity_search(query=query)

# building the dictionary for chain

chain_input={
    "input_documents": docs,
    "context":"This is contextless",
    "question":query,
    "Customer_Name":"Bob",
    "Customer_State":"NY",
    "Customer_Gender":"Male"
}

result=chain.invoke(chain_input, return_only_outputs=True)

In [117]:
result['output_text']

' "Did you find it?" asked Toad.'

In [118]:
query="What was my first question to you?"

docs = db.similarity_search(query=query)

# building the dictionary for chain

chain_input={
    "input_documents": docs,
    "context":"This is contextless",
    "question":query,
    "Customer_Name":"Bob",
    "Customer_State":"NY",
    "Customer_Gender":"Male"
}

result=chain.invoke(chain_input, return_only_outputs=True)

In [119]:
result['output_text']

'\nNo, said Frog.'